In [143]:
class Data:
    def __init__(self, data:str, ip:int):
        self.ip = ip
        self.data = data

class Server:
    LAST_IP = 1
    
    def __init__(self):
        self.ip = Server.LAST_IP
        Server.LAST_IP += 1
        self.router = None
        self.buffer = []
    
    def send_data(self, data:Data):
        self.router.buffer.append(data)
    
    def get_data(self):
        buffer = [packet for packet in self.buffer]
        self.buffer.clear()
        return buffer
    
    def get_ip(self):
        return self.ip

class Router:
    def __init__(self):
        self.links = {}
        self.buffer = []
    
    def link(self, server):
        server.router = self
        self.links[server.ip] = server
    
    def unlink(self, server):
        unlinked_server = self.links.pop(server.ip, False)
        if unlinked_server:
            unlinked_server.router = None
    
    def send_data(self):
        for data in self.buffer:
            if data.ip in self.links:
                server = self.links[data.ip]
                server.buffer.append(data)
        self.buffer.clear()

In [144]:
router = Router()
sv_from = Server()
sv_from2 = Server()
router.link(sv_from)
router.link(sv_from2)
router.link(Server())
router.link(Server())
sv_to = Server()
router.link(sv_to)
sv_from.send_data(Data("Hello", sv_to.get_ip()))
sv_from2.send_data(Data("Hello", sv_to.get_ip()))
sv_to.send_data(Data("Hi", sv_from.get_ip()))
router.send_data()
msg_lst_from = sv_from.get_data()
msg_lst_to = sv_to.get_data()

In [146]:
for msg in msg_lst_from:
    print("Router - ", msg.ip, 'Message: ', msg.data)
for msg in msg_lst_to:
    print("Router - ", msg.ip, 'Message: ', msg.data)

Router -  1 Message:  Hi
Router -  5 Message:  Hello
Router -  5 Message:  Hello
